In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("lab05.ipynb")

In [1]:
# Run this cell to load all dependencies 
exec(open("./utils.py").read())

# Lab 5 – Advanced Table Methods

## Data 6, Summer 2022

Today we will be exploring some more complex table methods we can use! The `apply`, `group`, `pivot`, and `join` methods all allow us to perform different queries on our familiar tables. Understanding not only *how* each method works, but also *why* and *when* to use them are the key takeaways from this lab; by the end of it, we will be able to query tables in some pretty cool ways!

These new methods allow us to do different operations than before. As such, it is becoming more and more important to remember how each method sits on our Data Science toolbelt. We should think of each new method as a **tool that serves a specific purpose**. Your job as a data scientist is not only to understand what each tool does, but when each tool is applicable in new situations!

In this lab, we'll be working with a `movies` data set that contains information about various American films over time. It contains the following columns:
1. `"Film"`: The name of the movie
2. `"Genre"`: The genre of the movie
3. `"Year"`: The year the movie was released
4. `"Lead Studio"`: The primary movie studio responsible for producing the movie
5. `"Audience score %"`: The score, out of 100%, given to the movie by viewers
6. `"Rotten Tomatoes %"`: The score, out of 100%, given to the movies by the website [Rotten Tomatoes](https://www.rottentomatoes.com/)
7. `"Worldwide Gross"`: The total gross revenue, in millions of dollars, that the movie made
8. `"Quality"`: Descriptive ranking of the movie based on audience score

Let's load the dataset into a new table, `movies`, to get practice with the more advanced table methods.

In [2]:
movies = Table.read_table("data/movies.csv")
movies.show(5)

## The [apply](http://data8.org/datascience/_autosummary/datascience.tables.Table.apply.html#datascience.tables.Table.apply) method

The `apply` method allows us to map a function's behavior onto an entire column of a table. We can use built-in Python functions (like `max`) or we can define our own functions and then *apply* those functions to the columns of a table.

The `apply` method takes at least 2 arguments. The first is a function, and the rest are as many column labels you need to run that function. The number of columns you need to specify is dependent on the number of arguments the function has. For example, if the function you provide needs two inputs, you need to list two columns for it to work on.

`apply` returns a NumPy array of the transformed values. We can ask questions like "How can I categorize the items in this column?" (like converting grade percentages into letter grades from lecture). We can also make modifications to a table, like rounding all the values in a column to a certain accuracy.

### Example Use
We can see some examples of `apply` at work. Let's use `apply` to take the average of the two score percentages, `"Audience score %"` and `"Rotten Tomatoes %"`.

In [3]:
def average_score(audience_score, rt_score):
    "Computes the average score between the audience score and Rotten Tomatoes score"
    return (audience_score + rt_score) / 2

In [4]:
average_scores = movies.apply(average_score, "Audience score %", "Rotten Tomatoes %")
average_scores

Now, let's add a new column called `"Average score %"` and populate it with the information we just assigned to `average_scores`. We'll re-assign this new table to `movies`.

In [5]:
movies = movies.with_column("Average score %", average_scores)
movies.show(5)

We can use `apply` in this way to add columns to a table that transforms something in the table to something that more people can understand more easily!

### Input Type and Output Type

Very quickly, we should talk about **input type** and **output type**. **Input type** is the *type* of the arguments in a function. A function can take in as input any of the types we have talked about so far in this class (int, str, boolean, etc.). The input type is very important when using the table methods we talk about today, because there can be behavior we do not expect if we give a function the wrong input type. Let's see an example of this behavior when we try to use our `average_score` function on a row that has an **input type** of `str` instead of `int`:

In [6]:
movies.apply(average_score, "Genre", "Lead Studio")

The error message does not make it *explicitly* clear that the function you provided got an incorrect input type, but if you are using a table method and the error message seems to indicate incompatible types interacting with each other, that may be the issue. You can see this on the **last line of the error message**:

```python
TypeError: unsupported operand type(s) for /: 'str' and 'int'
```

In simple terms, it means you're trying to *divide a string by an integer*, which you cannot do in Python.

**Question One**: Fill in the function `convert_to_dollars` which converts a dollar amount from *millions of dollars* to *dollars*. Then, use the `apply` method to convert all values from the `"Worldwide Gross (Millions)"` column into dollars. Finally, create a new column in the `movies` table called `"Worldwide Gross"` using the array resulting from your call to `apply`.

*Note*: The code for this question requires several steps. Feel free to create new cells to experiment!

<!--
BEGIN QUESTION
name: q1
points: 0
-->

In [7]:
def convert_to_dollars(dollar_millions):
    "Converts a dollar amount from millions of dollars to dollars"
    ...

dollars = ...
movies = ...
movies

In [ ]:
grader.check("q1")

## The [group](http://data8.org/datascience/_autosummary/datascience.tables.Table.group.html#datascience.tables.Table.group) method

The `group` method is very helpful for organizing a table before asking more questions about it. you can think of the `group` method as organizing rows into bins based on one of their values. All the rows that share a column value go in the same bin! We saw this in action in Lab 3 when we grouped tables in order to visualize certain variables like `"Month"` or `"Binge Drink"` in the BRFSS data set. Now, we will learn how to use the `group` method ourselves.

`group` takes in 1-2 arguments. The first is a column label to group by, and the second is an optional function argument to group on, which defaults to counting the number of rows in the bin. We will see some examples of how this optional argument works below.

Let's use the `group` method to organize our movies by `Genre` so that we can see which genres are in our table:

In [13]:
movies_by_genre = movies.group("Genre")
movies_by_genre

**Question 2**: What if we want to group by movie *quality*? Fill in the following cell with code that will assign `quality_groups` to a grouped table based on the `"Quality"` column.

<!--
BEGIN QUESTION
name: q2
points: 0
-->

In [14]:
quality_groups = ...
quality_groups

In [ ]:
grader.check("q2")

We can use the optional `collect` argument to ask more complex questions instead of just asking for the number of rows in each bin. For example, to see the average `"Audience Score %"` of each bin, we can use `np.mean` or `np.average`.

*Note*: For our purposes, `np.mean` and `np.average` are exactly the same and you can use them interchangeably in the call to `group`. However, the function you choose will impact the **column names** of the resulting table.

In [18]:
quality_group_ratings = movies.group("Quality", np.average)
quality_group_ratings

### Shortcomings of `collect` argument

Look at the `quality_group_rating` table above -- notice that we cannot specify which column we want the average of. As you can see, the `group` method will take the average of every column. For columns where it is **possible to take the average** (columns with the correct input type), it does so. For columns where taking the average doesn't work (with strings in this case), Python outputs nothing for those column averages.

Also, the column labels in this new table now have `average` at the end **except** the column label you grouped on. The name of the function you choose to group on will appear at the end of every label in the resulting table. **Like most table methods, the original table you are grouping does not change unless you reassign it**.

To get only the column we wanted the average of, we can use the `select` method to get the `Quality` and the `Audience score % average`:

In [19]:
quality_ratings = quality_group_ratings.select("Quality", "Audience score % average")
quality_ratings

**Question 3:** The table defaults to sort on the column you grouped on, which in this case is alphabetical order for strings. Oscar asks you to sort the `quality_ratings` table in decreasing order based on the average audience score. 

Write a line of code below that assigns `ratings_sorted` to a table that matches Oscar's request.

<!--
BEGIN QUESTION
name: q3
points: 0
-->

In [20]:
ratings_sorted = ...
ratings_sorted

In [ ]:
grader.check("q3")

**Question 4 (Do Well-Liked Movies Make Money?)**: After hearing that you have a handy `movies` table, Rebecca asks you the following question:
>*Do well-liked movies make money?*

It's your job to answer Rebecca's question. To do so, create a new table called `money_by_quality` with the following **two columns**:
1. `"Quality"`: String describing the quality of a given movie
2. `"Worldwide Gross (Millions) mean"`: The average gross revenue for each movie quality

*Hint*: You may find the previous calls to `group` helpful.

<!--
BEGIN QUESTION
name: q4
points: 0
-->

In [23]:
money_by_quality = ...
money_by_quality

In [ ]:
grader.check("q4")

Look at that! Using the `group` method, we found that **"Great"** movies make an average of 215 million dollars while **"Okay"** movies only make an average of 71.7 million dollars. Let's report back to Rebecca!

## The [pivot](http://data8.org/datascience/_autosummary/datascience.tables.Table.pivot.html#datascience.tables.Table.pivot) method

The `pivot` method allows us to see the *intersection* of two of our column labels. `pivot` essentially sorts the contents of the dataset based on the combination of the two column labels you pivot on. All the table's rows that share values in the pivoting columns all go into the same bin, and this happens for all combinations of the first and second column you are pivoting on.

The `pivot` method has 4 important arguments, 2 of which are mandatory and 2 of which are optional:

| **Argument** | **Description** |
| --- | --- |
| `columns` | The label whose unique values will appear as the **columns** of the outputted pivot table |
| `rows` | The label whose unique values will appear as the **rows** of the outputted pivot table |
| *Optional:* `values` | Values to use when aggregating |
| *Optional:* `collect` | Function used to aggregate the `values` provided in the previous argument |

You must use the two `values` and `collect` arguments together, one does not work without the other.

The best way to understand how `pivot` works is to look at some examples and talk about what happens. Here we will pivot the `movies` table on its `Genre` and `Quality` column labels to see how many of each type of *genre* are in each category of *quality*:

In [28]:
genre_quality_pivot = movies.pivot("Genre", "Quality")
genre_quality_pivot

The way we can read this table is almost how we read a graph in a math class. To see how many **Good Comedies** there are, we look at the row corresponding to `Good` and the column corresponding to `Comedy`, so there are 18 **Good Comedies**.

The default behavior of `pivot` is to just count the rows that appear at each intersection of the pivot. However, we can ask it to count another value in the table using any function we want! For example, if we want to know **the maximum amount of money** each intersection made instead of **how many** movies appear in each intersection, we can do that as well!

In [29]:
genre_quality_average_grosses = movies.pivot("Genre", "Quality", values="Worldwide Gross (Millions)", collect=max)
genre_quality_average_grosses

Based on this small dataset, it seems that the highest grossing film is not actually a "Great" film but rather a "Good" one. Run the following cell to find out what film it actually is!

In [30]:
movies.where("Worldwide Gross (Millions)", 709.82)

Ah, Twilight! Who can resist Robert Pattinson and Taylor Lautner?

**Question 5 (Does Studio Matter?)**: Now, let's write a query that can tell us what genre of movie each studio tends to make, and how much those movies made in total. We can use the `pivot` method just like we did above to see the breakdown of movies by studio and genre, and then aggregate those rows by adding up the worldwide gross of each intersection!

Assign `column_label` and `row_label` to column labels of `movies_with_qualities` that make the resulting `pivot` call have studios as the rows and genres as the columns.

Assign `value_to_collect` to a column label that we can use to collect the data we need to ultimately see the total amount of money made by each intersection of genre and studio.

Assign `collection_function` to a function you know that can add up all the movie earnings at each intersection

<!--
BEGIN QUESTION
name: q5
points: 0
-->

In [31]:
column_label = ...
row_label = ...
value_to_collect = ...
collection_function = ...

studio_genre_total_gross = movies.pivot(column_label, row_label, value_to_collect, collection_function)
studio_genre_total_gross

In [ ]:
grader.check("q5")

## The [join](http://data8.org/datascience/_autosummary/datascience.tables.Table.join.html#datascience.tables.Table.join) method

The last method we will talk about in this lab is the `join` method. This method allows us to combine two different tables together!

The `join` method takes in 2 mandatory arguments and 1 optional argument:

| **Column Name** | **Description** |
| --- | --- |
| `column_label` | a column to use to join |
| `other` | another table |
| *Optional:* `other_label` | `other`'s label to join on (if not the same as `column_label`) |

If `other` has a label in common with the table you are joining with and this common label is the one you want to join on, then you do not need to use the optional argument. If you want to join on another column label or if neither table has a column label in common, then you can use the optional `other_label`.

The way join works takes some getting used to, so let's look at some examples of `join` at work!

We have the `dogs` and `owners` tables below, take a look at them a bit before we move on so you understand what data they contain:

In [36]:
dogs = Table().with_columns(
    "Name", np.array(["Spot", "Rex", "Fluffy", "Doge"]),
    "Breed", np.array(["Golden Retriever", "Cockapoo", "Corgi", "Coin"]),
    "Owner", np.array(["James", "Will", "Josh", "Sandra"]),
)
dogs

In [37]:
owners = Table().with_columns(
    "Owner", np.array(["James", "Josh", "Sandra", "Will"]),
    "Owner Age", np.array([18, 21, 20, 21])
)
owners

As you can see, we have a column in common: `Owner`. Let's join these two tables together so that we can have all the doggy data in one place!

In [38]:
doggy_data = dogs.join("Owner", owners)
doggy_data

This table now has all of our information in one place, which makes using it easier!

Now let's take a look at a more common example, where the column labels being named differently can cause a problem. We will use the exact same `dogs` and `owners` tables, but we will change a column label on `owners`:

In [39]:
owners_new_label = owners.relabeled("Owner", "Name")

display(dogs, owners_new_label)

Now if we try to use the `join` method like we did last time, we run into an issue...

In [40]:
doggy_data = dogs.join("Owner", owners_new_label)
doggy_data

Because the `owners` table does not have a column label called `Owner`, we may try to use the one column label they do have in common: `Name`...

In [41]:
doggy_data = dogs.join("Name", owners_new_label)
doggy_data

...but this doesn't appear to work either. This is not an error, there is simply no table outputted by this join call!

This `join` call does not do what we want, because the `dogs` `Name` corresponds to the **dog's** name, but the `owners` `Name` corresponds to the **Owner's** name! No dog and owner have any of the same names, so there is no data to output in this `join` call!

Instead, we have to make sure we join on the `Owner` column from `dogs` and the `Name` column from `owners`! We can do this using the third *optional* argument in the `join` method:

In [42]:
doggy_data = dogs.join("Owner", owners_new_label, "Name")
doggy_data

In this table, the `Name` column now refers to the name of the dog, and the `Owner` column corresponds to the name of the owner!

**Question 6 (A Slightly Different `Join`)**: The `join` method can also change the number of rows in its output. If there are multiple rows in one table that match with one row in the other, the `join` method will include rows for each of these matches in the output. Also, if there is a row in a table with no match in the other, there will be no row in the output that represents this row. Let's implement both these situations in practice and see how they work:

In [43]:
# Just run this cell
# This new dogs table has a new extra dogs
more_dogs = dogs.with_rows(make_array(make_array("Clifford", "Big Red", "Sandra"), make_array("Doug", "Golden Retriever", "Russell")))
more_dogs

**Task**: Before we exectute the join between these tables, we should be able to calculate how many rows should there be in the output. Assign the variable `more_dog_owner_rows` to the number of rows that should result from joining `more_dogs` with `owners_new_label`. Run the cell below to see them again for clarity:

In [44]:
display(more_dogs, owners_new_label)

In [45]:
more_dog_owner_rows = ...
more_dog_owner_rows

In [ ]:
grader.check("q6")

### Confirm your answer

Remember, each row in the `more_dogs` table only gets a row in the output if it matches a row in `owners_new_label`

In [48]:
# If an owner in the table has 2 dogs, both dogs should appear in the output
# If a dog has no owner in the owners table, the dog does not appear in the output
complex_doggy_data = more_dogs.join("Owner", owners_new_label, "Name")
complex_doggy_data

In [49]:
# Doug is not in the new table, and both Sandra's dogs are present for a total of 5 rows
complex_doggy_data.num_rows

As we expect, both `Spot` and `Clifford` appear as Sandra's dogs, but `Doug` does not appear in the table (fear not, he still has an owner, but `Russell` is not in the owners table).

## Done! 😇

That's it! There's nowhere for you to submit this, as labs are not assignments. However, please ask any questions you have with this notebook in lab or on Ed.

There are no extra problems this week, good luck with the homework!

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False)